In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from random import random, sample
from scipy import stats
import copy 
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

#regression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

#evaluation metrics
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score


In [2]:
import lightgbm as lgb

In [3]:
data = pd.read_csv('D:/Dataset/arkavidia/final/data_training.csv')
df_test = pd.read_csv('D:/Dataset/arkavidia/final/test.csv')

## Praproses

In [4]:
data

,departure_day,departure_time_hour,order_id,payment_day,payment_week,departure_week,account_id,adult,child,infant,age,gender,customer_price
0,Thursday,17:40,dc4b93999e67112bd673b5457a4052b8,Tuesday,9,22,071bb7304b7e2de18e8d5efc6fbe2bbe,1,0,0,30,F,1090570.0
1,Thursday,05:00,addf13cf9693c22f291484dbcab53bbd,Saturday,12,22,114ff495b211d2d5eb057fcc3c90f971,1,0,0,0,Unknown,1090570.0
2,Thursday,17:40,8f8455d559593e5842532c68fbfb832d,Tuesday,9,22,071bb7304b7e2de18e8d5efc6fbe2bbe,1,0,0,30,F,1090570.0
3,Thursday,17:40,96b9666be21d0ac77acb12b449145ed4,Sunday,11,22,5dea1945219bddda6b16459241df0c12,2,0,0,58,M,2181140.0
4,Thursday,17:40,9af805a69bf77d35465bc59728c375d2,Sunday,11,22,5dea1945219bddda6b16459241df0c12,2,0,0,58,M,2181140.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8391,Wednesday,12:25,bf758b68b67682898854f157c7f62f64,Tuesday,21,36,7c8d009573c685c6a356d0751b499c92,1,2,0,0,F,3844425.0
8392,Thursday,12:25,ceb03ddb15f25a430cef00dd61634c3a,Monday,29,36,8c58438bc6555bcfa54d589f1f802b8a,3,0,0,0,F,6571890.0
8393,Sunday,12:25,d5e9b1022383fd82e75e234e5225e65f,Friday,33,36,a16d4258f510940ca7d8c60d672a78b8,1,2,0,0,M,6672250.0
8394,Thursday,08:45,24043c4c67aa6907c97629e6742875bd,Sunday,28,36,f3c774d1fd2983aff1482d9af084ddaa,3,0,0,0,Unknown,5124990.0


In [5]:
df_test = df_test[['departure_week', 'departure_day', 'departure_time_hour']]
df_test

,departure_week,departure_day,departure_time_hour
0,34,Monday,5:00
1,34,Monday,8:45
2,34,Monday,12:25
3,34,Tuesday,5:00
4,34,Tuesday,8:45
...,...,...,...
85,36,Saturday,14:55
86,36,Sunday,5:00
87,36,Sunday,5:50
88,36,Sunday,12:25


In [6]:
string = 'Jumlah missing value pada {}'
print(string.format('data flight'), '\n', data.isnull().sum(), '\n', '\n',
      string.format('data test'), '\n', df_test.isnull().sum())

Jumlah missing value pada data flight 
 departure_day            0
departure_time_hour      0
order_id                 0
payment_day              0
payment_week             0
departure_week           0
account_id               0
adult                    0
child                    0
infant                   0
age                      0
gender                 217
customer_price           0
dtype: int64 
 
 Jumlah missing value pada data test 
 departure_week         0
departure_day          0
departure_time_hour    0
dtype: int64


In [7]:
string = 'Jumlah baris unik tiap variabel pada {}'
print(string.format('data flight'), '\n', data.nunique(), '\n', '\n', 
      string.format('data test'),'\n',df_test.nunique())

Jumlah baris unik tiap variabel pada data flight 
 departure_day             7
departure_time_hour       6
order_id               8245
payment_day               7
payment_week             30
departure_week           15
account_id             7000
adult                     7
child                     5
infant                    3
age                      77
gender                    3
customer_price          552
dtype: int64 
 
 Jumlah baris unik tiap variabel pada data test 
 departure_week         3
departure_day          7
departure_time_hour    6
dtype: int64


In [8]:
data['total_penumpang'] = data.adult + data.child + data.infant
data

,departure_day,departure_time_hour,order_id,payment_day,payment_week,departure_week,account_id,adult,child,infant,age,gender,customer_price,total_penumpang
0,Thursday,17:40,dc4b93999e67112bd673b5457a4052b8,Tuesday,9,22,071bb7304b7e2de18e8d5efc6fbe2bbe,1,0,0,30,F,1090570.0,1
1,Thursday,05:00,addf13cf9693c22f291484dbcab53bbd,Saturday,12,22,114ff495b211d2d5eb057fcc3c90f971,1,0,0,0,Unknown,1090570.0,1
2,Thursday,17:40,8f8455d559593e5842532c68fbfb832d,Tuesday,9,22,071bb7304b7e2de18e8d5efc6fbe2bbe,1,0,0,30,F,1090570.0,1
3,Thursday,17:40,96b9666be21d0ac77acb12b449145ed4,Sunday,11,22,5dea1945219bddda6b16459241df0c12,2,0,0,58,M,2181140.0,2
4,Thursday,17:40,9af805a69bf77d35465bc59728c375d2,Sunday,11,22,5dea1945219bddda6b16459241df0c12,2,0,0,58,M,2181140.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8391,Wednesday,12:25,bf758b68b67682898854f157c7f62f64,Tuesday,21,36,7c8d009573c685c6a356d0751b499c92,1,2,0,0,F,3844425.0,3
8392,Thursday,12:25,ceb03ddb15f25a430cef00dd61634c3a,Monday,29,36,8c58438bc6555bcfa54d589f1f802b8a,3,0,0,0,F,6571890.0,3
8393,Sunday,12:25,d5e9b1022383fd82e75e234e5225e65f,Friday,33,36,a16d4258f510940ca7d8c60d672a78b8,1,2,0,0,M,6672250.0,3
8394,Thursday,08:45,24043c4c67aa6907c97629e6742875bd,Sunday,28,36,f3c774d1fd2983aff1482d9af084ddaa,3,0,0,0,Unknown,5124990.0,3


In [9]:
data['departure_time_hour'] = data['departure_time_hour'].map({'05:00':0, '05:50':1, '08:45':2, 
                                                               '12:25':3, '14.55':4, '17.40':5})
df_test['departure_time_hour'] = df_test['departure_time_hour'].map({'5:00':0, '5:50':1, '8:45':2, 
                                                               '12:25':3, '14:55':4, '17:40':5})

In [10]:
string = 'Jumlah missing value pada {}'
print(string.format('data flight'), '\n', data.isnull().sum(), '\n', '\n',
      string.format('data test'), '\n', df_test.isnull().sum())

Jumlah missing value pada data flight 
 departure_day             0
departure_time_hour    2081
order_id                  0
payment_day               0
payment_week              0
departure_week            0
account_id                0
adult                     0
child                     0
infant                    0
age                       0
gender                  217
customer_price            0
total_penumpang           0
dtype: int64 
 
 Jumlah missing value pada data test 
 departure_week         0
departure_day          0
departure_time_hour    0
dtype: int64


In [11]:
data

,departure_day,departure_time_hour,order_id,payment_day,payment_week,departure_week,account_id,adult,child,infant,age,gender,customer_price,total_penumpang
0,Thursday,NaN,dc4b93999e67112bd673b5457a4052b8,Tuesday,9,22,071bb7304b7e2de18e8d5efc6fbe2bbe,1,0,0,30,F,1090570.0,1
1,Thursday,0.0,addf13cf9693c22f291484dbcab53bbd,Saturday,12,22,114ff495b211d2d5eb057fcc3c90f971,1,0,0,0,Unknown,1090570.0,1
2,Thursday,NaN,8f8455d559593e5842532c68fbfb832d,Tuesday,9,22,071bb7304b7e2de18e8d5efc6fbe2bbe,1,0,0,30,F,1090570.0,1
3,Thursday,NaN,96b9666be21d0ac77acb12b449145ed4,Sunday,11,22,5dea1945219bddda6b16459241df0c12,2,0,0,58,M,2181140.0,2
4,Thursday,NaN,9af805a69bf77d35465bc59728c375d2,Sunday,11,22,5dea1945219bddda6b16459241df0c12,2,0,0,58,M,2181140.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8391,Wednesday,3.0,bf758b68b67682898854f157c7f62f64,Tuesday,21,36,7c8d009573c685c6a356d0751b499c92,1,2,0,0,F,3844425.0,3
8392,Thursday,3.0,ceb03ddb15f25a430cef00dd61634c3a,Monday,29,36,8c58438bc6555bcfa54d589f1f802b8a,3,0,0,0,F,6571890.0,3
8393,Sunday,3.0,d5e9b1022383fd82e75e234e5225e65f,Friday,33,36,a16d4258f510940ca7d8c60d672a78b8,1,2,0,0,M,6672250.0,3
8394,Thursday,2.0,24043c4c67aa6907c97629e6742875bd,Sunday,28,36,f3c774d1fd2983aff1482d9af084ddaa,3,0,0,0,Unknown,5124990.0,3


In [12]:
df_test

,departure_week,departure_day,departure_time_hour
0,34,Monday,0
1,34,Monday,2
2,34,Monday,3
3,34,Tuesday,0
4,34,Tuesday,2
...,...,...,...
85,36,Saturday,4
86,36,Sunday,0
87,36,Sunday,1
88,36,Sunday,3


In [13]:
def tot_penumpang(week, day, jam, daymin1):
    a = data[data['departure_week'] == week]
    b = a[a.departure_day == day]
    c = b[b.departure_time_hour == jam]
    d = c[c.payment_week >= week]
    e = d[d.payment_day == day]
    f = d[d.payment_day == daymin1]

    print(
            'Week', week, day, jam, '\n'
            'Total Penumpang =', c.total_penumpang.sum(), '\n'
            'Bayarnya pas mau Terbang =', e.total_penumpang.sum()+f.total_penumpang.sum()
         )
    return c.total_penumpang.sum(), e.total_penumpang.sum()+f.total_penumpang.sum()

In [14]:
waktu = [0, 1, 2, 3, 4, 5]
hari = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
minggu = [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]

isi = []

for week in minggu:
    for day in hari:
        index = hari.index(day)
        daymin1 = hari[index-1]
        for time in waktu:
            f, g = tot_penumpang(week, day, time, daymin1)
            if f > 0:
                isi.append([week, day, time, f, g])
            print('\n')

Week 22 Monday 0 
Total Penumpang = 47 
Bayarnya pas mau Terbang = 0


Week 22 Monday 1 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 22 Monday 2 
Total Penumpang = 41 
Bayarnya pas mau Terbang = 1


Week 22 Monday 3 
Total Penumpang = 51 
Bayarnya pas mau Terbang = 0


Week 22 Monday 4 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 22 Monday 5 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 22 Tuesday 0 
Total Penumpang = 54 
Bayarnya pas mau Terbang = 3


Week 22 Tuesday 1 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 22 Tuesday 2 
Total Penumpang = 47 
Bayarnya pas mau Terbang = 8


Week 22 Tuesday 3 
Total Penumpang = 33 
Bayarnya pas mau Terbang = 5


Week 22 Tuesday 4 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 22 Tuesday 5 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 22 Wednesday 0 
Total Penumpang = 37 
Bayarnya pas mau Terbang = 3


Week 22 Wednesday 1 
Total Penumpang = 59 
Bayarnya pas mau Terbang = 9





Week 25 Monday 5 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 25 Tuesday 0 
Total Penumpang = 6 
Bayarnya pas mau Terbang = 1


Week 25 Tuesday 1 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 25 Tuesday 2 
Total Penumpang = 3 
Bayarnya pas mau Terbang = 1


Week 25 Tuesday 3 
Total Penumpang = 18 
Bayarnya pas mau Terbang = 2


Week 25 Tuesday 4 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 25 Tuesday 5 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 25 Wednesday 0 
Total Penumpang = 1 
Bayarnya pas mau Terbang = 1


Week 25 Wednesday 1 
Total Penumpang = 2 
Bayarnya pas mau Terbang = 0


Week 25 Wednesday 2 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 25 Wednesday 3 
Total Penumpang = 13 
Bayarnya pas mau Terbang = 0


Week 25 Wednesday 4 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 25 Wednesday 5 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 25 Thursday 0 
Total Penumpang = 16 
Bayarnya pas mau Terb

Week 28 Wednesday 2 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 28 Wednesday 3 
Total Penumpang = 61 
Bayarnya pas mau Terbang = 11


Week 28 Wednesday 4 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 28 Wednesday 5 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 28 Thursday 0 
Total Penumpang = 47 
Bayarnya pas mau Terbang = 15


Week 28 Thursday 1 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 28 Thursday 2 
Total Penumpang = 35 
Bayarnya pas mau Terbang = 4


Week 28 Thursday 3 
Total Penumpang = 33 
Bayarnya pas mau Terbang = 2


Week 28 Thursday 4 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 28 Thursday 5 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 28 Friday 0 
Total Penumpang = 35 
Bayarnya pas mau Terbang = 6


Week 28 Friday 1 
Total Penumpang = 27 
Bayarnya pas mau Terbang = 5


Week 28 Friday 2 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 28 Friday 3 
Total Penumpang = 43 
Bayarnya pas mau T

Week 31 Tuesday 1 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 31 Tuesday 2 
Total Penumpang = 15 
Bayarnya pas mau Terbang = 8


Week 31 Tuesday 3 
Total Penumpang = 20 
Bayarnya pas mau Terbang = 7


Week 31 Tuesday 4 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 31 Tuesday 5 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 31 Wednesday 0 
Total Penumpang = 10 
Bayarnya pas mau Terbang = 4


Week 31 Wednesday 1 
Total Penumpang = 27 
Bayarnya pas mau Terbang = 9


Week 31 Wednesday 2 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 31 Wednesday 3 
Total Penumpang = 52 
Bayarnya pas mau Terbang = 22


Week 31 Wednesday 4 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 31 Wednesday 5 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 31 Thursday 0 
Total Penumpang = 26 
Bayarnya pas mau Terbang = 17


Week 31 Thursday 1 
Total Penumpang = 0 
Bayarnya pas mau Terbang = 0


Week 31 Thursday 2 
Total Penumpang = 20 
Bayarnya pas 

In [15]:
df = pd.DataFrame(isi, columns = ['departure_week', 'departure_day', 'departure_time_hour', 'tot_penumpang', 'nofood'])
df.to_csv("forecast.csv", index=False)
df

,departure_week,departure_day,departure_time_hour,tot_penumpang,nofood
0,22,Monday,0,47,0
1,22,Monday,2,41,1
2,22,Monday,3,51,0
3,22,Tuesday,0,54,3
4,22,Tuesday,2,47,8
...,...,...,...,...,...
259,33,Saturday,2,43,13
260,33,Saturday,3,52,15
261,33,Sunday,0,5,2
262,33,Sunday,1,24,6


In [16]:
df['departure_day'] = df['departure_day'].map({'Monday':0, 'Tuesday':1, 'Wednesday':2, 
                                                               'Thursday':3, 'Friday':4, 'Saturday':5, 'Sunday':6})
df_test['departure_day'] = df_test['departure_day'].map({'Monday':0, 'Tuesday':1, 'Wednesday':2, 
                                                               'Thursday':3, 'Friday':4, 'Saturday':5, 'Sunday':6})

In [17]:
string = 'Jumlah missing value pada {}'
print(string.format('data flight'), '\n', df.isnull().sum(), '\n', '\n',
      string.format('data test'), '\n', df_test.isnull().sum())

Jumlah missing value pada data flight 
 departure_week         0
departure_day          0
departure_time_hour    0
tot_penumpang          0
nofood                 0
dtype: int64 
 
 Jumlah missing value pada data test 
 departure_week         0
departure_day          0
departure_time_hour    0
dtype: int64


In [18]:
X = df[['departure_day', 'departure_week', 'departure_time_hour']]
y = df.nofood

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [20]:
reg = XGBRegressor(max_depth = 4, learning_rate = 0.313, n_estimator = 1)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
y_pred = np.absolute(y_pred)
rmsle_pred = mean_squared_log_error(y_test, y_pred) 

print("Root Mean Squared Log Error:" , np.sqrt(rmsle_pred))

[15:17:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Root Mean Squared Log Error: 1.1714165711265478


In [21]:
models=[XGBRegressor(), RandomForestRegressor(),AdaBoostRegressor(),BaggingRegressor(),SVR(),KNeighborsRegressor()]
model_names=['XGBRegressor', 'RandomForestRegressor','AdaBoostRegressor','BaggingRegressor','SVR','KNeighborsRegressor']
rmsle=[]
d={}
for model in range (len(models)):
    clf=models[model]
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    y_pred = np.absolute(y_pred)
    rmsle.append(np.sqrt(mean_squared_log_error(y_pred,y_test)))
d={'Modelling Algo':model_names,'RMSLE':rmsle}   
d

[15:17:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


{'Modelling Algo': ['XGBRegressor',
  'RandomForestRegressor',
  'AdaBoostRegressor',
  'BaggingRegressor',
  'SVR',
  'KNeighborsRegressor'],
 'RMSLE': [1.0205232564625975,
  1.027771255629977,
  1.0969524490457498,
  1.0342470743331424,
  1.0263313719567377,
  1.073668979202356]}

In [22]:
modelLR = LinearRegression()
modelLR.fit(X_train, y_train)
y_pred = modelLR.predict(X_test)
y_pred = np.absolute(y_pred)
rmsle_pred = mean_squared_log_error(y_test, y_pred) 

print("Root Mean Squared Log Error:" , np.sqrt(rmsle_pred))

Root Mean Squared Log Error: 1.183270292763722


In [23]:
modelSVR = SVR(gamma=1, C=1, epsilon = 0.14)
modelSVR.fit(X_train, y_train)
y_pred = modelSVR.predict(X_test)
y_pred = np.absolute(y_pred)
rmsle_pred = mean_squared_log_error(y_test, y_pred) 

print("Root Mean Squared Log Error:" , np.sqrt(rmsle_pred))

Root Mean Squared Log Error: 0.9646241591708287


In [66]:
reg = XGBRegressor(max_depth = 10, learning_rate = 0.01, n_estimator = 10, gamma = 21, 
                   max_delta_step=10, min_child_weight=0.8)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
y_pred2 = np.absolute(y_pred)
rmsle_pred = mean_squared_log_error(y_test, y_pred2) 

print("Root Mean Squared Log Error:" , np.sqrt(rmsle_pred))

[15:25:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Root Mean Squared Log Error: 0.9089900001563066


In [26]:
modelSVR = SVR(gamma=4, C=1, epsilon = 0)
modelSVR.fit(X_train, y_train)
y_pred = modelSVR.predict(X_test)
y_pred0 = np.absolute(y_pred)
rmsle_pred = mean_squared_log_error(y_test, y_pred) 

print("Root Mean Squared Log Error:" , np.sqrt(rmsle_pred))

Root Mean Squared Log Error: 1.0422565712047667


In [27]:
reg1 = XGBRegressor(max_depth = 100, learning_rate = 0.01, n_estimator = 100, gamma = 21, objective='reg:linear')
reg1.fit(X_train, y_train)
y_pred = reg1.predict(X_test)
y_pred1 = np.absolute(y_pred)
rmsle_pred = mean_squared_log_error(y_test, y_pred1) 

print("Root Mean Squared Log Error:" , np.sqrt(rmsle_pred))

[15:18:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Root Mean Squared Log Error: 0.9378872137675337


In [34]:
y_pred = 0.1*y_pred0 + 0.1*y_pred1 + 0.8*y_pred2
rmsle_pred = mean_squared_log_error(y_test, y_pred) 

print("Root Mean Squared Log Error:" , np.sqrt(rmsle_pred))

Root Mean Squared Log Error: 0.9380649348631416


In [76]:
df_test = df_test[['departure_day', 'departure_week', 'departure_time_hour']]

In [77]:
indeks = np.arange(0, 90)
prediction = reg.predict(df_test)
prediction = np.array(prediction)
submission = pd.DataFrame({
        "index": indeks,
        "total_pax": prediction
    })
submission.to_csv("submission.csv", index=False)

In [ ]:
df

In [ ]:
df_test